In [1]:
!git clone https://github.com/nkt780426/Bachelor-s-Project.git

Cloning into 'Bachelor-s-Project'...
remote: Enumerating objects: 694, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 694 (delta 0), reused 3 (delta 0), pack-reused 686 (from 1)
Receiving objects: 100% (694/694), 110.88 MiB | 18.28 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [ ]:
!mv Bachelor-s-Project/going_modular .

In [3]:
!ls -la

total 20
drwxr-xr-x 5 root root 4096 Dec 15 09:13 .
drwxr-xr-x 5 root root 4096 Dec 15 09:12 ..
drwxr-xr-x 2 root root 4096 Dec 15 09:12 .virtual_documents
drwxr-xr-x 5 root root 4096 Dec 15 09:12 Bachelor-s-Project
drwxr-xr-x 8 root root 4096 Dec 15 09:12 going_modular


In [1]:
!rm -rf checkpoint/

In [2]:
import os

import torch
from torchinfo import summary
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchvision import transforms
from torchvision.transforms import InterpolationMode

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.learningrate import NormalMapLR
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'depthmap',
    'train_dir': '/kaggle/input/normap-map/depthmap/Depth_Map/train',
    'test_dir': '/kaggle/input/normap-map/depthmap/Depth_Map/test',
    
    # Cấu hình train
    'backbone': 'iresnet18',
    'epochs': 500,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('/kaggle/input/normap-map/depthmap/Depth_Map/train')),
    'embedding_size': 512,
    
    'learning_rate': 2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

train_transform = transforms.Compose([
    transforms.RandomCrop(224),
])

test_transform = transforms.Compose([
    transforms.Resize((224,224), interpolation=InterpolationMode.NEAREST)
])

train_dataloader, val_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = NormalMapLR(optimizer=optimizer, fixed_epoch=2, fixed_lr=20, start_warm_lr=2)
checkpoint_path = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/' + CONFIGURATION['backbone'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/' + CONFIGURATION['backbone'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['val_euclidean_accuracy', 'val_cosine_accuracy', 'val_auc_euclidean', 'val_auc_cosine'],
    patience=40,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=50
)

In [ ]:
fit(CONFIGURATION, 0, model, device, train_dataloader, val_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 1:
	train: loss 3.470 | loss id   3.45 | top_1_acc 0.0082 | top_5_acc 0.0299 | acc_eu: 0.430 | acc_cos: 0.440 | auc_eu: 0.546 | auc_cos: 0.521
	val: acc_eu: 0.625 | acc_cos: 0.672 | auc_eu: 0.581 | auc_cos: 0.568
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/depthmap/iresnet18/models/checkpoint.pth
Epoch 2:
	train: loss 2.729 | loss id   2.71 | top_1_acc 0.0060 | top_5_acc 0.0262 | acc_eu: 0.684 | acc_cos: 0.308 | auc_eu: 0.549 | auc_cos: 0.529
	val: acc_eu: 0.658 | acc_cos: 0.414 | auc_eu: 0.581 | auc_cos: 0.559
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/depthmap/iresnet18/models/checkpoint.pth
Epoch 3:
	train: loss 0.457 | loss id   0.43 | top_1_acc 0.0045 | top_5_acc 0.0329 | acc_eu: 0.552 | acc_cos: 0.551 | auc_eu: 0.560 | auc_cos: 0.561
	val: acc_eu: 0.636 | acc_cos: 0.628 | auc_eu: 0.598 | auc_cos: 0.602
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/depthmap/iresnet18/models/checkpoint.pth
Epoc

In [65]:
!ls -la checkpoint/magface/depthmap/iresnet18/models

total 316120
drwxr-xr-x 2 root root      4096 Dec 15 15:48 .
drwxr-xr-x 4 root root      4096 Dec 15 15:35 ..
-rw-r--r-- 1 root root  46257324 Dec 15 15:35 best_val_auc_cosine_210.pth
-rw-r--r-- 1 root root  46257769 Dec 15 15:35 best_val_auc_euclidean_206.pth
-rw-r--r-- 1 root root  46258023 Dec 15 15:35 best_val_cosine_accuracy_215.pth
-rw-r--r-- 1 root root  46258404 Dec 15 15:35 best_val_euclidean_accuracy_147.pth
-rw-r--r-- 1 root root 138654838 Dec 15 15:35 checkpoint.pth


In [64]:
!rm checkpoint/magface/depthmap/iresnet18/models/best_val_auc_cosine_100.pth

In [66]:
!zip -r output.zip checkpoint

  adding: checkpoint/ (stored 0%)
  adding: checkpoint/magface/ (stored 0%)
  adding: checkpoint/magface/depthmap/ (stored 0%)
  adding: checkpoint/magface/depthmap/iresnet18/ (stored 0%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/ (stored 0%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/Cosine_Accuracy_train/ (stored 0%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/Cosine_Accuracy_train/events.out.tfevents.1734254720.8ee67753c5a0.23.5 (deflated 67%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/Train_accuracy_id_top_1/ (stored 0%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/Train_accuracy_id_top_1/events.out.tfevents.1734254720.8ee67753c5a0.23.1 (deflated 70%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/Euclidean_Accuracy_val/ (stored 0%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/Euclidean_Accuracy_val/events.out.tfevents.1734254720.8ee67753c5a0.23.4 (deflated 69%)
  adding: checkpoint/magface/depthmap/iresnet18/logs/AUC_co